### Load Packages, Constants and Format Functions
the Python code based on Tom O'Shea's C++ code (arXiv: 2406.01691), and made lots of modifications

In [3]:
import os
import numpy as np
from scipy.constants import pi
from math import log, exp, sqrt, pow

# constants
pi = 3.14159265359
alpha = 1 / 137.035999084  # fine structure constant
me = 510998.950  # e- mass [eV]
Mpl = 2e27  # planck mass [eV]
zeta3 = 1.202056903159594  # Riemann zeta(3)
amu = 931.494e6  # amu in eV

# conversion factors
s2eV = 6.582119569e-16  # Hz to eV
J2eV = 1.0 / 1.602176634e-19  # Joules to eV (1 / e)
m2eV = 1.973269804e-7  # m-1 to eV
K2eV = 8.617333262e-5  # Kelvin to eV
kg2eV = 5.609588604e35
T2eV = 2e2 # Tesla to eV2 conversion [eV2/T]

In [4]:
# define functions for read, write2D, readGaunt, writeREST
def read(filename):
    """Reads a file and returns a list of floats."""
    try:
        with open(filename, 'r') as file:
            return [float(line.strip()) for line in file if line.strip()]
    except FileNotFoundError:
        print(f"Error: File {filename} not found.")
        return [69.]

def write2D(filename, data1, data2):
    """Writes two lists of equal length to a file as two columns."""
    if len(data1) != len(data2):
        print("ERROR - ensure vectors are of equal length")
        return
    
    if os.path.exists(filename):
        os.remove(filename)
        print(f"File {filename} deleted.")
    
    with open(filename, 'w') as file:
        for d1, d2 in zip(data1, data2):
            file.write(f"{d1}\t{d2}\n")
    print(f"File {filename} created successfully.")

def readGaunt(filename):
    """Reads a space-delimited file and returns a 2D list of floats."""
    try:
        with open(filename, 'r') as file:
            g2D = []
            for i, line in enumerate(file):
                row = [float(x) for x in line.split()]
                if i == 0:
                    row.insert(0, 0.0)  # Add a zero at the beginning of the first line
                g2D.append(row)
                if len(g2D) == 501:
                    break
            return g2D
    except FileNotFoundError:
        print(f"Couldn't find file: {filename}")
        return [[69.]]

def writeREST(filename, data, line):
    """Writes a list of data values to a file, appending or overwriting as needed."""
    if line == 0 and os.path.exists(filename):
        os.remove(filename)
        print(f"File {filename} deleted.")
    
    with open(filename, 'a') as file:
        file.write("\t".join(map(str, data)) + "\n")
    
    if line == 0:
        print(f"Writing to file {filename}...\n")

### Solar Parameters and Solar Models

In [5]:
# solar parameters
dSolar = 149.5978707e9 / m2eV  # mean earth-sun distance [eV-1]
rSolar = 6.957e8 / m2eV  # solar radius [eV-1]
B0 = 3e3 * T2eV  # radiative zone max B [eV2]  200*T2eV;
B1 = 50 * T2eV  # tachocline max B [eV2]  4*T2eV;//
B2 = 3 * T2eV  # outer region max B [eV2]  3*T2eV;//
r0 = 0.712 * rSolar  # [eV-1]
r1 = 0.732 * rSolar  # [eV-1]
d1 = 0.02 * rSolar  # [eV-1]
r2 = 0.96 * rSolar  # [eV-1]
d2 = 0.035 * rSolar  # [eV-1]
ngamma0 = 1e25 * m2eV * m2eV * s2eV  # photon flux at r0 [eV3]


# solar model
file = "/home/yuangw/Documents/GitHub/Chameleons/chameleon_2406.01691/data/"
ne = read(file + "ne.dat")  # electron number density [eV3]
nbar = read(file + "nbar.dat")  # Z2-summed number density [eV3]
nbar2 = read(file + "nbar2.dat")  # Z2-summed number density minus electrons [eV3]
wp = read(file + "wp.dat")  # plasma frequency [eV]
T = read(file + "T.dat")  # solar temperature [eV]
r = read(file + "r.dat")  # radial distance [eV-1]
rho = read(file + "rho.dat")  # solar density [eV4]
nH = read(file + "nH.dat")  # H number density [eV3]
nHe3 = read(file + "nHe3.dat")  # He3 number density [eV3]
nHe4 = read(file + "nHe4.dat")  # He4 number density [eV3]
z1 = readGaunt(file + "Z1.dat")  # gaunt factors for Z=1
z2 = readGaunt(file + "Z2.dat")  # gaunt factors for Z=2

### Chameleon Model

In [6]:
# cham model params n (phi-n potential), Bm (matter coupling), assume rho dominated by matter density
# units ev2
def mCham2(c, Bm):  # effective mass square, Eq.18 in 2406.01691
    """Chameleon mass squared as a function of solar radius."""
    E4n = pow(E, 4 + n)
    if n < 0:
        return (
            n
            * (n + 1)
            * E4n
            * pow(pow(Bm * rho[c] / (n * Mpl * E4n), (n + 2)), 1 / (n + 1))
        )
    else:
        return n * (n + 1) * E4n * pow(Bm * rho[c] / (n * Mpl * E4n), (n + 2) / (n + 1))


### Magnetic Field Production

In [7]:
def GammaPhoton(w, c, g1, g2):  # paper 2406.01691 Eq.A20

    p1 = 64 * pow(pi, 2) * pow(alpha, 3)
    p2 = 3 * pow(me, 2) * pow(w, 3)
    p3 = me * pow(ne[c], 2) / (2 * pi * T[c])
    p4 = 1 - exp(-w / T[c])
    p5 = 8 * pi * pow(alpha, 2) * ne[c] / (3 * pow(me, 2))

    # sum of ion densities
    ions = (nH[c] * g1) + g2 * ((4 * nHe4[c]) + (4 * nHe3[c]))

    return p1 * pow(p2, -1) * pow(p3, 0.5) * p4 * ions + p5


def selectG(c, w):
    """Selects Gaunt factor from matrix and computes Gamma (eV)."""
    indexT1 = indexT2 = indexX1 = indexX2 = 0

    # Find indices for temperature interpolation
    for i in range(1, 200):
        if z1[0][i] < T[c] < z1[0][i + 1]:
            indexT1 = i
        if z2[0][i] < T[c] < z2[0][i + 1]:
            indexT2 = i

    # Find indices for frequency interpolation
    for i in range(1, 500):
        if (z1[i][0] * T[c]) < w < (z1[i + 1][0] * T[c]):
            indexX1 = i
        if (z2[i][0] * T[c]) < w < (z2[i + 1][0] * T[c]):
            indexX2 = i

    g1 = z1[indexT1][indexX1]
    g2 = z2[indexT2][indexX2]
    return GammaPhoton(w, c, g1, g2)


def Bfield(c):
    """Solar magnetic field as a function of radius."""
    if r[c] <= r0:  # B-field in solar radiative zone
        l = (10 * (r0 / rSolar)) + 1
        K = (1 + l) * pow(1 + pow(l, -1), l) * B0
        return K * pow(r[c] / r0, 2) * pow(1 - pow(r[c] / r0, 2), l)
    elif r1 - d1 < r[c] < r1 + d1:  # B-field in tachocline
        return B1 * (1 - pow((r[c] - r1) / d1, 2))
    elif r2 - d2 < r[c] < r2 + d2:  # B-field in outer region
        return B2 * (1 - pow((r[c] - r2) / d2, 2))
    else:
        return 0


def B_integrand(c, Bm, w):  # [eV Bg-2] Eq.7 in paper 2406.01691
    """Differential scalar production rate due to magnetic field."""
    if T[c] == 0:
        return 0

    mg2 = 4 * pi * alpha * ne[c] / me
    ms2 = mCham2(c, Bm)
    B = Bfield(c)
    G = selectG(c, w)

    if w**2 <= mg2 or w**2 <= ms2:
        return 0

    return (2/ (pi * Mpl**2) * r[c] ** 2* B**2 * w * pow(w**2 - ms2, 1.5) / ((ms2 - mg2) ** 2 + (w * w * G * G)) * G/ (exp(w / T[c]) - 1))  


def B_solarIntg(w, Bm, r, ne, T, rho, Mpl, B0, B1, B2, r0, r1, d1, r2, d2):
    """Given scalar mass and energy, Integral over solar volume for magnetic field production."""
    total = 0
    for c in range(len(r) - 1):
        total += ( 0.5* (r[c + 1] - r[c])* (B_integrand(c + 1, Bm, w) + B_integrand(c, Bm, w)) )
    return total


### Electron-Ion Primakoff

In [8]:
# dimensionless, with dimensionless arguments
def curlyI(u, v):  # Eq.37 in 2406.01691
    """Integral I(u,v) solved analytically incross section calc"""
    return ( (u * u - 1) / v * log((u - 1) / (u + 1))- (pow(u + v, 2) - 1) / v * log((u + v - 1) / (u + v + 1))- 2 )


def curlyIapprox(u, v):  # I(u,v) in u=>1 limit
    return u * u / v - (v + 2) * log(v / (v + 2)) - 2


def T_integrand(c, Bm, w):
    """Differential scalar production rate on Earth d2N/dr/dw divided by beta_gamma^2"""
    if T[c] == 0:
        return 0  # solve weird behaviour when ne=T=0

    mg2 = 4 * pi * 7.297e-3 * ne[c] / 0.511e6  # wp2 = 4 pi alpha ne / me [eV^2]
    ms2 = mCham2(c, Bm)

    if w**2 <= mg2 or w**2 <= ms2:
        return 0

    K2 = 8 * pi * 7.297e-3 * nbar[c] / T[c]  # Debye screening scale^2 [eV^2]
    kgamma = sqrt(w**2 - mg2)
    kphi = sqrt(w**2 - ms2)

    uArg = kgamma / (2 * kphi) + kphi / (2 * kgamma)
    vArg = K2 / (2 * kphi * kgamma)
    Iuv = curlyI(uArg, vArg)
    if uArg < 1.01:
        Iuv = curlyIapprox(uArg, vArg)

    return (alpha/ (8 * Mpl**2 * pi)* r[c] ** 2* nbar[c] / (exp(w / T[c]) - 1) * w**2 * kphi/ kgamma* Iuv)


def T_solarIntg(w, Bm):
    """For a given scalar mass and energy, Integral over solar volume."""
    total = 0
    for c in range(len(r) - 1):
        total += (0.5 * (r[c + 1] - r[c]) * (T_integrand(c + 1, Bm, w) + T_integrand(c, Bm, w)))
    return total

### Photon Coalescence

In [9]:
## Appendix B: Coalescence-Primakoff equivalence
def integrand_ll(c, Bm):
    """LL Coalescence, Differential scalar production rate on earth dN/dr times Lambda2"""
    if T[c] == 0:
        return 0

    ms2 = mCham2(c, Bm)
    mg2 = 4 * pi * alpha * ne[c] / me
    kgamma = sqrt(w * w - mg2)
    K2 = 8 * pi * alpha * ne[c] / T[c]

    if 2 * mg2 <= ms2:
        return 0

    return (1/ (8 * Mpl * Mpl * pi * pi) * kgamma * kgamma * T[c] * T[c] * r[c] * r[c] * sqrt(4 * mg2 - ms2))
    # return 1/(36 * Mpl * Mpl * pi * pi) * pow(K2, 1.5) * T[c] * T[c] * r[c] * r[c] * sqrt(4 * mg2 - ms2)  //* pow(exp(kgamma/wp[c])-1, -1)


def integrand_lt(c, Bm, w):
    """LT Coalescence, Differential scalar production rate on earth d2N/dr/dw times Lambda2"""
    if T[c] == 0:
        return 0

    ms2 = mCham2(c, Bm)
    mg2 = 4 * pi * alpha * ne[c] / me

    if w**2 <= mg2 or w**2 <= ms2:
        return 0

    wt = w - sqrt(mg2)  # t-photon omega [eV2]
    kphi = sqrt(w**2 - ms2)  # scalar wavenumber [eV]
    kt = sqrt(w * (w - 2 * sqrt(mg2)))  # t-photon wavenumber [eV]

    if w - 2 * sqrt(mg2) <= 0:
        return 0

    return (2* r[c] ** 2/ (Mpl**2 * 12 * pi**2)* wt**2 * kphi * kt * T[c]/ (exp(wt / T[c]) - 1))


def integrand_decay(c, Bm, w):
    """Differential scalar production rate for plasmon decay, d2N/dr/dw times Lambda2"""
    if T[c] == 0:
        return 0

    mg2 = 4 * pi * alpha * ne[c] / me
    ms2 = mCham2(c, Bm)

    if w**2 <= mg2 or w**2 <= ms2:
        return 0

    wt = w + sqrt(mg2)  # t-photon omega [eV]
    kphi = sqrt(w**2 - ms2)  # scalar wavenumber [eV]
    kt = sqrt(w * (w - 2 * sqrt(mg2)))  # t-photon wavenumber [eV]

    if w - 2 * sqrt(mg2) <= 0:
        return 0

    return (r[c] ** 2/ (12 * Mpl**2 * pi**2)* wt**2* kphi* kt* T[c]/ (exp(wt / T[c]) - 1))


def solarIntg_lt(w, Bm):
    """Integral over solar volume for LT and decay contributions"""
    total = 0
    for c in range(len(r) - 1):
        total += ( 0.5* (r[c + 1] - r[c])* (integrand_lt(c + 1, Bm, w)+ integrand_lt(c, Bm, w)+ integrand_decay(c + 1, Bm, w)+ integrand_decay(c, Bm, w)))
    return total  # returns dPhi/dw Bg-2 [eV2]


def kIntg_ll(c, Bm):
    """Integral over k_gamma for l-plasmon"""
    total = 0
    kMax = sqrt(2 * me * wp[c])
    dk = kMax / 1000
    for k in np.arange(dk, kMax, dk):
        total += 0.5 * dk * (integrand_ll(c, Bm, k + dk) + integrand_ll(c, Bm, k))
    return total


def A(y, u):  # A(y, u) for coalescence, dimensionless
    return pow(u - y, 2) * log((u + 1) / (u - 1)) - 2 * u + 4 * y


def integrand_ll_omega(c, Bm, w, w1):  # LL coalescence with w1 INTG
    """Differential scalar production rate on Earth d2N/dr/dk times Lambda2"""
    if T[c] == 0:
        return 0

    if w1 < wp[c] or w < 2 * wp[c]:
        return 0

    ms2 = mCham2(c, Bm)  # Chameleon mass2 [eV2]
    kphi = sqrt(w * w - ms2)  # phi momentum [eV]
    kgamma = sqrt(me / 3 / T[c] * (w**2 - wp[c] ** 2))
    K2 = 8 * pi * alpha * ne[c] / T[c]  # Debye screening scale^2 [eV2]

    yArg = kgamma / kphi
    uArg = yArg / 2 + 1 / yArg / 2
    Ayu = A(yArg, uArg)
    w2 = w - w1

    return (1/ (16 * Mpl * Mpl * pi * pi)* me / 3/ T[c]* r[c] ** 2* kphi**2 * w1**2/ (exp(w1 / T[c]) - 1)* w2/ (exp(w2 / T[c]) - 1)* Ayu )


def w1Intg(c, w, Bm):
    """Integral over all w1, returns dN/dr [eV^2]."""
    total = 0
    w1 = wp[c]

    while True:
        I1 = integrand_ll_omega(c, Bm, w, w1 + 1)
        I0 = integrand_ll_omega(c, Bm, w, w1)

        if I1 + I0 == 0:
            break

        total += 0.5 * (I0 + I1)
        w1 += 1

    return total  # [eV^2]


def solarIntg_ll_omega(w, Bm):
    """Integral over solar volume for a given scalar mass and energy, returns N Bg-2 [eV]."""
    total = 0
    for c in range(len(r) - 1):
        total += 0.5 * (r[c + 1] - r[c]) * (w1Intg(c + 1, w, Bm) + w1Intg(c, w, Bm))
    return total  # [eV Bg-2]



### Comparision with CAST Limits

In [10]:
def CAST_old():
    """get old CAST back-converted flux for old limits"""
    beta, flux = [], []
    model = "CAST_old"
    Emin, Emax = 1e3, 2e4
    B_cast = 9 * T2eV  # CAST B-field [eV2]
    L_cast = 9.26 / m2eV  # CAST length  [eV-1]
    P = (B_cast * L_cast / (2 * Mpl)) ** 2  # back conversion prob for low m
    dw = 10

    for Bm in np.logspace(-1, 8, num=10):
        total = sum(
            0.5 * dw * P * (B_solarIntg(w + dw, Bm) + B_solarIntg(w, Bm))
            for w in np.arange(Emin, Emax, dw)
        )
        beta.append(Bm)
        flux.append(total / (4 * pi * dSolar**2 * ((Emax - Emin) / 1e3)))
        print(f"Bm = {Bm} of 1e8")

    write2D(f"data/{model}_totalflux.dat", beta, flux)


def CAST_new():
    beta, flux = [], []
    model = "CAST_new"
    Emin, Emax = 1e3, 2e4
    B_cast = 9 * T2eV  # CAST B-field [eV2]
    L_cast = 9.26 / m2eV  # CAST length  [eV-1]
    P = (B_cast * L_cast / (2 * Mpl)) ** 2  # back conversion prob for low m
    dw = 10

    for Bm in np.logspace(-1, 8, num=10):
        total = sum(
            0.5 * dw * P * (T_solarIntg(w + dw, Bm) + T_solarIntg(w, Bm))
            for w in np.arange(Emin, Emax, dw)
        )
        beta.append(Bm)
        flux.append(total / (4 * pi * dSolar**2 * ((Emax - Emin) / 1e3)))
        print(f"Bm = {Bm} of 1e8")

    write2D(f"data/{model}_totalflux.dat", beta, flux)


def Ibrax(a):
    """Brax Integral"""
    return sqrt(pi / 2) * (sqrt(a + sqrt(a**2 + 4)) - sqrt(2 * a))


def Brax(c, Bm, w):
    """Brax tachocline calculation, differential emission rate d2N/dw/dr"""
    if not (r1 / rSolar - 0.05 <= r[c] / rSolar <= r1 / rSolar + 0.05):
        return 0

    mfp_t = 0.3 / 100 / m2eV  # tachocline mean free path [eV-1]
    flux_t = 1e21 * 1e4 * m2eV**2 * s2eV  # tachocline photon flux [eV3]
    n_t = 2 * zeta3 * T[c] ** 3 / (pi**2)  # tachocline photon density [eV3]
    mg2 = 4 * pi * alpha * ne[c] / me  # assume mg2 = wp2 [eV2]
    ms2 = mCham2(c, Bm)

    if w**2 <= mg2 or w**2 <= ms2:
        return 0

    kgamma = sqrt(w**2 - mg2)
    kphi = sqrt(w**2 - ms2)
    q = np.abs(kgamma - kphi)
    B = Bfield(c)

    return (2 /(pi * Mpl**2) *(r[c] ** 2 * B**2) /(exp(w / T[c]) - 1)*(w**4 / mg2**2) *flux_t/ (2 * pi * n_t * mfp_t)* sqrt(q /(2 * s2eV))*Ibrax(2 /(mfp_t * q)))


def solarIntgBrax(w, Bm):
    """Integral over solar volume for a given scalar mass and energy"""
    return sum( 0.5 * (r[c + 1] - r[c]) * (Brax(c + 1, Bm, w) + Brax(c, Bm, w)) for c in range(len(r) - 1) if r1 / rSolar - 0.05 <= r[c] / rSolar <= r1 / rSolar + 0.05
    )


def CAST_Brax():
    """Get old CAST back-converted flux for old limits"""
    beta, flux = [], []
    model = "CAST_Brax"
    Emin, Emax = 1e3, 2e4
    B_cast = 9 * T2eV  # babyIAXO B-field [eV2]
    L_cast = 9.26 / m2eV  # babyIAXO length  [eV-1]
    P = (B_cast * L_cast / (2 * Mpl)) ** 2
    dw = 1e1

    for Bm in np.logspace(-1, 8, num=10):
        total = sum(
            0.5 * dw * P * (solarIntgBrax(w + dw, Bm) + solarIntgBrax(w, Bm))
            for w in np.arange(Emin, Emax, dw)
        )
        beta.append(Bm)
        flux.append(total / (4 * pi * dSolar**2 * ((Emax - Emin) / 1e3)))
        print(f"Bm = {Bm} of 1e8")

    write2D(f"data/{model}_totalflux.dat", beta, flux)

### Checking Bg Assumptions

In [11]:
# Chameleon mass as a function of solar radius and model parameters
def mCham2_B(c, Bg):
    if n == 0:
        raise ValueError("ERROR! n = 0")
    E4n = E ** (4 + n)
    B = B0
    return (n* (n + 1)* E4n* ((2 * Bm * rho[c] + Bg * B * B) / (2 * n * Mpl * E4n)) ** ((n + 2) / (n + 1)))


# Differential scalar production rate on Earth d2N/dr/dw divided by beta_gamma^2
def T_integrand_B(c, Bm, w):
    if T[c] == 0:
        return 0  # Solves weird behavior when ne = T = 0

    mg2 = 4 * pi * alpha * ne[c] / me  # Assume mg2 = wp2
    ms2 = mCham2_B(c, Bm)  # Chameleon mass squared [eV2]

    if w**2 <= mg2 or w**2 <= ms2:
        return 0

    K2 = 8 * pi * alpha * nbar[c] / T[c]  # Debye screening scale squared [eV2]
    kgamma = sqrt(w**2 - mg2)  # Photon momentum [eV]
    kphi = sqrt(w**2 - ms2)  # Scalar momentum [eV]
    uArg = kgamma / (2 * kphi) + kphi / (2 * kgamma)  # u for curlyI
    vArg = K2 / (2 * kphi * kgamma)  # v for curlyI
    Iuv = curlyI(uArg, vArg)

    # Explicitly put in the u => 1 limit to avoid numerical issues
    if uArg < 1.01:
        Iuv = curlyIapprox(uArg, vArg)

    return ((alpha / (8 * Mpl * Mpl * pi))* (r[c] ** 2)* nbar[c] / (exp(w / T[c]) - 1)* w**2* kphi/ kgamma * Iuv)  # [eV Bg-2]


# Integral over solar volume, for a given scalar mass and energy
# Returns dN/dw Bg-2, Units: Bg-2
def T_solarIntg_B(w, Bm):
    total = 0
    for c in range(len(r) - 1):
        total += ( 0.5* (r[c + 1] - r[c])* (T_integrand_B(c + 1, Bm, w) + T_integrand_B(c, Bm, w)))
    return total  # [Bg-2]

In [12]:
### Plotting and Bits

In [13]:
# calculate emission rate profile over solar radius (dN/dr)
# integrated over relevant energies up to 20 keV
def profile(option, r):
    radius, rate = [], []
    Bm = 100
    n, dw = 1, 1
    for c in range(len(r)):
        total = 0
        for w in np.arange(dw, 2e4, dw):
            if option == "T":
                total += 0.5 * dw * (T_integrand(c, Bm, w + dw) + T_integrand(c, Bm, w))
            elif option == "B":
                total += 0.5 * dw * (B_integrand(c, Bm, w + dw) + B_integrand(c, Bm, w))
        radius.append(r[c])
        rate.append(total)
    name = f"data/{option}_profile_1e2.dat"
    write2D(name, radius, rate)


# calculate differential particle flux spectrum dN/dw by intg over solar volume
def spectrum(option, Bm=1e2, n=1, dw=1e0):
    count, energy = [], []
    Bm = 100
    n, dw = 1, 1
    for w in np.arange(dw, 2e4, dw):
        energy.append(w)
        if option == "T":
            count.append(T_solarIntg(w, Bm))
        elif option == "B":
            count.append(B_solarIntg(w, Bm))
        if int(w) % int(1e3) == 0:
            print(f"w = {w/1e3} keV of 20 keV")
    name = f"data/{option}_spectrum_1e2.dat"
    write2D(name, energy, count)


# calculate total energy loss rate as funtion of Bm
# only T-dominate contribution
def Eloss(n=1, dw=1e2):
    mass, Q = [], []
    dw, n = 100, 1
    for Bm in np.logspace(-1, 4, num=6):
        total = sum(
            0.5 * dw * ((w + dw) * T_solarIntg(w + dw, Bm) + w * T_solarIntg(w, Bm))
            for w in np.arange(dw, 2e4, dw)
        )
        mass.append(Bm)
        Q.append(total)
        print(f"Bm = {Bm}")
    write2D("data/Eloss_Bm_T.dat", mass, Q)


# calculate total energy loss rate as a function of Bg
def Eloss_Bg():
    mass, Q = [], []
    Bm = 100
    dw, n = 10, 1
    Bm = 1e2
    w1, w2, r1, r2 = 0, 0, 0, 0
    Bg = 1e-1
    while Bm <= 1e20:
        total = 0
        for w in np.arange(dw, 2e4, dw):
            total += (0.5 * dw * ((w + dw) * T_solarIntg_B(w + dw, Bg) + w * T_solarIntg_B(w, Bg)))
        mass.append(Bg)
        Q.append(total)
        print(f"Bg = {Bg}")
        Bm *= 10

    name = "data/Eloss_Bg--constB.dat"
    write2D(name, mass, Q)


# calculate energy loss as a function of n
def Eloss_n():
    nvec, Q = [], []
    dw = 1e1
    Bm = 1e2
    n = -10
    w1, w2, r1, r2 = 0, 0, 0, 0
    while n <= 100:
        if n < 0 and int(n) % 2 != 0:
            # n += 4
            continue
        total = 0
        for w in np.arange(dw, 2e4, dw):
            total += (0.5 * dw * ((w + dw) * T_solarIntg(w + dw, Bm) + w * T_solarIntg(w, Bm)))
        nvec.append(n)
        Q.append(total)
        print(f"n = {n}")
        n += 4

    name = "data/Eloss_n--1e2.dat"
    write2D(name, nvec, Q)


# calculate energy loss as a function of Lambda
def Eloss_Lambda():
    Evec, Q = [], []
    dw, Bm = 1e1, 1e6
    n, E = 1, 1e-8
    while E <= 1e1:
        total = 0
        for w in np.arange(dw, 2e4, dw):
            total += ( 0.5 * dw * ((w + dw) * T_solarIntg(w + dw, Bm) + w * T_solarIntg(w, Bm)) )
        Evec.append(E)
        Q.append(total)
        print(f"E = {E}")
        E *= 1.1

    name = "data/Eloss_Lambda_T--1e6.dat"
    write2D(name, Evec, Q)


# chameleon mass squared as a function of solar radius for given model parameters
def mass_profile():
    radius, mass = [], []
    Bm = 1e2  # cham matter coupling
    n = 1  # cham model n
    E = 2.4e-3  # Lambda [eV]
    for c in range(len(r)):
        radius.append(r[c])
        mass.append(mCham2(c, Bm))

    # write to file
    name = "data/mass_profile_1e2.dat"
    write2D(name, radius, mass)


# get m as a function of Lambda and B_m for given n
def mass_contour():
    mass, BOut, LOut = [], [], []
    mB, mL, n = 1.1, 1.1, 1
    c = 0
    name = "data/massregion_n1--1e3.dat"

    Bm = 1e-1
    while Bm <= 1e18:
        BOut.append(Bm)
        L = 1e-8
        while L <= 1e1:
            E = L
            total = sqrt(mCham2(0, Bm))
            total = min(total, 1e3)
            mass.append(total)
            if c == 0:
                LOut.append(E)
            L *= mL
        writeREST(name, mass, c)
        mass.clear()
        c += 1
        Bm *= mB

    write2D("data/massregion_Bm.dat", BOut, BOut)
    write2D("data/massregion_Lambda.dat", LOut, LOut)
    print("\acompleted it mate")


# calculate differential particle flux spectrum by intg over solar volume
# LT coalescence and plasmon decay, dN/dw
def spectrum_lt():
    count, energy = [], []
    Bm = 1e2
    n, dw = 1, 1
    for w in np.arange(dw, 2e4, dw):
        energy.append(w)
        count.append(solarIntg_lt(w, Bm))
        if int(w) % int(1e3) == 0:
            print(f"w = {w / 1e3} keV of 20 keV")

    name = "data/coalescence_lt_spectrum_1e2--uncapped.dat"
    write2D(name, energy, count)


def profile_ll():
    radius, rate = [], []
    Bm, n = 1e2, 1
    for c in range(len(r)):
        radius.append(r[c])
        rate.append(kIntg_ll(Bm, c))

    name = "data/coalescence_ll_profile_1e2.dat"
    write2D(name, radius, rate)


# calculate differential  particle flux spectrum by intg over solar volume
def spectrum_ll():
    count, energy = [], []
    Bm, n = 1e2, 1
    w2, r2 = 0, rSolar
    for j in reversed(range(len(wp))):
        w1 = wp[j]
        if w2 > w1:
            continue
        r1 = r[j]
        energy.append(2 * w1)
        count.append(kIntg_ll(Bm, j) * abs((r2 - r1) / (w2 - w1)) / 2)
        r2, w2 = r[j], wp[j]

    name = "data/coalescence_ll_spectrum_1e2--test.dat"
    write2D(name, energy, count)


# ll spectrum for omega intg
def spectrum_ll_omega():
    count, energy = [], []
    Bm, n = 1e2, 1
    dw = 1.0
    for w in np.arange(dw, 2e4, dw):
        energy.append(w)
        count.append((solarIntg_ll_omega(w + dw, Bm) - solarIntg_ll_omega(w, Bm)) / dw)

    name = "data/coalescence_ll_spectrum_omega.dat"
    write2D(name, energy, count)


def main():
    # Convert Gaunt factor Theta to T in eV
    for i in range(1, 201):
        z1[0][i] *= me
    for i in range(1, 201):
        z2[0][i] *= me

    spectrum("T")


if __name__ == "__main__":
    main()


NameError: name 'E' is not defined